In [ ]:
import utils.dataset_utils as dataset
import utils.train_utils as train

import numpy as np
import copy
import pickle

from pykalman import KalmanFilter
import matplotlib.pyplot as plt
import preprocessing.GSP as GSP


plt.rcParams['figure.figsize'] = (20.0, 10.0)
plt.rcParams['figure.dpi'] = 500


In [ ]:
def unit_vector_norm(X):
    return (X.T / np.sqrt((X**2).sum(axis=1))).T

In [ ]:
file_location = "../data/Raman_Mouse/corrected_4_wavenumbers/"
filenames = np.load(f"{file_location}FileNames.npy")
with open(f'{file_location}Sample_labels.pickle', 'rb') as f:
    labels = pickle.load(f)

wavenumbers = np.load(f"{file_location}Wavenumbers.npy")
    
data = []
for f in filenames:
    x = np.load(f"{file_location}{f.split('.')[0]}.npy")
    data.append(((x.reshape(-1,x.shape[-1])).reshape(x.shape), labels[f]))

In [ ]:
import numpy.polynomial.polynomial as poly
from scipy import ndimage

def smoothing(X, smooth=5, transition=10, spike_width=7):
    """
    Only remove noise from low noise to signal area's to maintain the intensity of the spikes.
    Noise is removed with a gaussian filter in spectral dimension.
    """
    grad = ndimage.gaussian_filter(X, (0, 1), order=1)
    grad_abs = np.abs(grad)
    grad_abs_sm = ndimage.gaussian_filter(grad_abs, (0, 5))
    mean_grad = np.mean(grad_abs, 1) + 1 / np.std(grad_abs, 1) * 3
    
    spikes = ((grad_abs_sm.T > mean_grad ).astype(float)).T 
    spikes = np.round(ndimage.gaussian_filter(spikes, (0, spike_width)))
    spikes = ndimage.uniform_filter(spikes, (0, transition))
    
    mean_grad = np.mean(grad_abs, 1)
    
    s = ((grad_abs_sm.T > mean_grad ).astype(float)).T 
    s = np.round(ndimage.gaussian_filter(s, (0, spike_width)))
    s = ndimage.uniform_filter(s, (0, transition))
      
    plt.plot(spikes[10])
    plt.plot(s[10])
    
    return (1 - spikes) * ndimage.gaussian_filter(X, (0,smooth)) + spikes * X


x = data[0][0].reshape(-1, data[0][0].shape[-1])

y = smoothing(x)

In [ ]:
for j, (f, img) in enumerate(zip(filenames, data)):   
    img = img[0]
    tmp = copy.copy(img.reshape(-1, img.shape[-1]))
    tmp1, tmp2 = GSP.split_Raman_af(GSP.smoothing(tmp, smooth=5, transition=10, spike_width=7), wavenumbers, 9)
#     tmp3, tmp4 = GSP.split_Raman_af(GSP.kalman_smoother(tmp), wavenumbers, 9)
    print(img.shape)
    for i in range(0, tmp1.shape[0], 43):
        x = tmp[i]
        kf = KalmanFilter(initial_state_mean=0, n_dim_obs=1)
        kf.em(x)
        kf.initial_state_mean = x[0]
        kf.observation_covariance *= 0.8
        y1 = kf.smooth(x)[0].flatten()

        kf.observation_covariance *= 20
        y2 = kf.smooth(x)[0].flatten()

        spikes = (np.abs(y1-y2) > np.mean(np.abs(y1-y2))*2).astype(float)
        spikes = np.round(ndimage.gaussian_filter(spikes, spike_width))
        spikes = ndimage.uniform_filter(spikes, transition)
        smooth = (1 - spikes) * y2 + spikes * y1
        print("splitting")
        raman = GSP.split_Raman_af(smooth.reshape(1,-1), wavenumbers, 9)[0].flatten()
        
        plt.plot(tmp[i], label='raw')
        plt.plot(tmp1[i], label='raman GSP')
#         plt.plot(tmp2[i], label='photoluminescence')
#         plt.plot(tmp3[0], label='photoluminescence2')
        plt.plot(smooth, label='smooth')
        plt.plot(raman, label='raman kalman')
#         plt.plot(tmp4[0]+tmp3[0], label='smooth2')
#         plt.plot(tmp[i]-tmp1[i]-tmp2[i]-0.01, label='error')
#         plt.plot(tmp[i]-tmp3[0]-tmp4[0]-0.01, label='error2')
        plt.legend()
        plt.show()
    break
    
    
#     np.save(f'{file_location2}{f.split("/")[-1].split(".")[0]}_raman', tmp1.reshape(shape))
#     np.save(f'{file_location2}{f.split("/")[-1].split(".")[0]}_photoluminescence', tmp2.reshape(shape))
    print(f"image: {f} is done.")
    

In [ ]:
obs = data[0][0][10,10]
kf = KalmanFilter(initial_state_mean=0, n_dim_obs=1)
kf.em(obs[::26])
kf.initial_state_mean = obs[0]
y = kf.smooth(obs)[0].flatten()
y = kf.smooth(y)[0].flatten()
plt.plot(wavenumbers, obs)
plt.plot(wavenumbers, y)
plt.show()

In [ ]:
from scipy import ndimage

# def smoothing(x, smooth=5, transition=10, spike_width=7):
#     """
#     Only remove noise from low noise to signal area's to maintain the intensity of the spikes.
#     Noise is removed with a gaussian filter in spectral dimension.
#     """
#     kf = KalmanFilter(initial_state_mean=0, n_dim_obs=1)
#     kf.em(x)
#     kf.initial_state_mean = obs[0]
#     kf.observation_covariance *= 30
#     y = kf.smooth(x)[0].flatten()
    
#     grad = ndimage.gaussian_filter(X, (0, 1), order=1)
#     grad_abs = np.abs(grad)
#     grad_abs_sm = ndimage.gaussian_filter(grad_abs, (0, 5))
#     mean_grad = np.mean(grad_abs, 1) + 1 / np.std(grad_abs, 1) * 3
    
#     spikes = ((grad_abs_sm.T > mean_grad ).astype(float)).T 
#     spikes = np.round(ndimage.gaussian_filter(spikes, (0, spike_width)))
#     spikes = ndimage.uniform_filter(spikes, (0, transition))
    
#     return (1 - spikes) * ndimage.gaussian_filter(X, (0,smooth)) + spikes * X

obs = data[0][0][10,10]
kf = KalmanFilter(initial_state_mean=0, n_dim_obs=1)

x = obs
kf.em(x)
kf.initial_state_mean = obs[0]
kf.observation_covariance *= 0.8
y = kf.smooth(obs)[0].flatten()
# y = kf.smooth(y)[0].flatten()
plt.plot(wavenumbers, obs)
plt.plot(wavenumbers, y)
plt.show()

In [ ]:
from scipy.interpolate import interp1d

def smooth(x):
    kf = KalmanFilter(initial_state_mean=0, n_dim_obs=1)
    kf.em(x)
    kf.initial_state_mean = x[0]
    kf.observation_covariance *= 0.8
    y1 = kf.smooth(x)[0].flatten()

    kf.observation_covariance *= 20
    y2 = kf.smooth(x)[0].flatten()

    diff = ndimage.gaussian_filter(y1-y2, 3)
    spikes_max = diff > np.mean(diff) + np.std(diff)
    grad = np.gradient(diff)

    pos_grad = grad > 0
    maximums = np.pad(pos_grad[2:]^pos_grad[:-2], (1,), 'constant', constant_values=(0,0))

    maximums_index = np.where(maximums)[0]
    spikes_index = np.where(spikes_max & maximums)[0].reshape(-1,2)[:,0]
    spikes = np.zeros(obs.shape)
    for L in spikes_index:
        i = np.where(maximums_index < L)[0][-1]
        spikes[range(maximums_index[i], maximums_index[i+3]+1)] = 1

    f = interp1d(wavenumbers[(1-spikes).astype(bool)], x[(1-spikes).astype(bool)], kind='linear')
    return f(wavenumbers)

obs = data[0][0][10,10]
x = smooth(obs)
plt.plot(wavenumbers, obs)
plt.plot(wavenumbers, x)
plt.show()

plt.plot(wavenumbers, x)
for _ in range(5):
    x = smooth(x)
    plt.plot(wavenumbers, obs)
    plt.plot(wavenumbers, x)
    plt.show()

kf = KalmanFilter(initial_state_mean=0, n_dim_obs=1)
kf.em(x)
kf.initial_state_mean = x[0]
kf.observation_covariance *= 5
y = kf.smooth(x)[0].flatten()

plt.plot(wavenumbers, obs)

# plt.plot(wavenumbers, spikes*2000)
plt.plot(wavenumbers, y)
# plt.plot(wavenumbers, y1)
# plt.plot(wavenumbers, y2)
# plt.plot(spikes*2000)
# plt.plot(wavenumbers, smooth)
# plt.plot(y1)
# plt.plot(np.abs(y1-y2))
# plt.plot([0,1300],[np.mean(np.abs(y1-y2))*2]*2)
plt.show()


In [ ]:
import numpy.polynomial.polynomial as poly

f2 = poly.polyfit(wavenumbers, y, 15)
z = poly.polyval(wavenumbers, f2)
plt.plot(wavenumbers, obs)
plt.plot(wavenumbers, z)

In [ ]:
# plt.plot(obs-z)
kf = KalmanFilter(initial_state_mean=0, n_dim_obs=1)
raman1 = (obs-z).clip(min=0)
plt.plot(obs)
plt.plot(raman1)
plt.plot(z)
plt.show()

In [ ]:
x = raman1
kf = KalmanFilter(initial_state_mean=0, n_dim_obs=1)
kf.em(x)
kf.initial_state_mean = x[0]
kf.observation_covariance *= 0.8
y1 = kf.smooth(x)[0].flatten()

kf.observation_covariance *= 20
y2 = kf.smooth(x)[0].flatten()

diff = ndimage.gaussian_filter(y1-y2, 3)
spikes_max = diff > np.mean(diff) + 2*np.std(diff)
grad = np.gradient(diff)

pos_grad = grad > 0
maximums = np.pad(pos_grad[2:]^pos_grad[:-2], (1,), 'constant', constant_values=(0,0))

maximums_index = np.where(maximums)[0]
spikes_index = np.where(spikes_max & maximums)[0].reshape(-1,2)[:,0]
spikes = np.zeros(obs.shape)
for L in spikes_index:
    i = np.where(maximums_index < L)[0][-1]
    spikes[range(maximums_index[i], maximums_index[i+3]+1)] = 1

spikes = ndimage.uniform_filter(spikes, 7)
raman2 = (1 - spikes) * y2 + spikes * y1    
raman2[raman2<np.max(raman2)/20] = 0

plt.plot(obs)
plt.plot(raman2)
plt.plot(z)
plt.show()

In [ ]:
x = obs-raman2
kf = KalmanFilter(initial_state_mean=0, n_dim_obs=1)
kf.em(x)
kf.initial_state_mean = x[0]
kf.observation_covariance *= 10
photo = kf.smooth(x)[0].flatten()

plt.plot(obs, label='raw')
plt.plot(photo, label='photoluminescence')
plt.plot(raman2, label='raman')
plt.plot(obs-photo-raman2, label='noise')
plt.legend()
plt.show()

In [ ]:
dir(kf)

In [ ]:
kf.observation_matrices, kf.transition_matrices

In [ ]:
kf.observation_covariance, kf.transition_covariance